In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('eda_data.csv')

#choose relevant columns
df.columns
df_columns = df[['Rating', 'Size','Type of ownership', 'Industry','Sector', 'Revenue','num_comp',
              'job_state', 'same_state','employer_provided', 'hourly','age', 'python_yn','spark', 'aws', 'excel',
              'job_simp', 'seniority','avg_salary','job_desc']]

#get dummy data
df_dum = pd.get_dummies(df_columns)
df_dum

from sklearn.model_selection import train_test_split
X = df_dum.drop('avg_salary', axis=1)
Y = df_dum.avg_salary.values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, random_state=42)

#train test split- train, validate, test
import statsmodels.api as sm

X_sm = X =sm.add_constant(X)
model = sm.OLS(Y,X_sm)
model.fit().summary()

#multiple linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
lm = LinearRegression()
lm.fit(X_train, Y_train)
cross_val_score(lm, X_train, Y_train, scoring = 'neg_mean_absolute_error', cv=3)
np.mean(cross_val_score(lm, X_train, Y_train, scoring = 'neg_mean_absolute_error', cv=3))

#lasso regression
from sklearn.linear_model import Lasso
lm_l = Lasso(alpha=.13)
lm_l.fit(X_train,Y_train)
np.mean(cross_val_score(lm_l, X_train, Y_train, scoring = 'neg_mean_absolute_error', cv=3))

alpha = []
error = []
for i in range(1,100):
    alpha.append(i/10)
    lml = Lasso(alpha=(i/10))
    error.append(np.mean(cross_val_score(lml, X_train, Y_train, scoring = 'neg_mean_absolute_error', cv=3)))
    
plt.plot(alpha, error)
err = tuple(zip(alpha, error))
df_err = pd.DataFrame(err, columns= ['alpha', 'error'])
df_err[df_err.error == max(df_err.error)]

#random forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

np.mean(cross_val_score(rf, X_train, Y_train, scoring = 'neg_mean_absolute_error', cv=3))

#tune models using GridSearchCV
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':range(10,300,10),'criterion':('mse','mae'), 'max_features':('auto','sqrt','log2')}
gs = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv=3)
gs.fit(X_train, Y_train)

gs.best_score_

gs.best_estimator_

#test ensemble
#use these different models to see predict the test set data
tpred_lm = lm.predict(X_test)
tpred_lml = lm_l.predict(X_test)
tpred_rf = gs.best_estimator_.predict(X_test)

from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test, tpred_lm)


mean_absolute_error(Y_test, tpred_lml)


mean_absolute_error(Y_test, tpred_rf)

(tpred_lm+tpred_rf)/2

mean_absolute_error(Y_test, (tpred_lm+tpred_rf)/2)

